# 1

As bibliotecas necessárias são importadas. A biblioteca Numpy para manipulação numérica, a biblioteca Pandas para criar os dataframes, os geradores de relatório Seaborn, os modelos de algoritmo de classificação pedidos no exercício: Regressão Logística, Arvore de Decisão, RandomForest, KNeighbors, Gradient Boosting, uma biblioteca para pré-processamento do dataset (StandardScaler), a biblioteca para medir a qualidade preditiva dos modelos (accuracy_score, confusion_matrix e classification_report) e a biblioteca para aplicar o Scaler para pré-processamento dos dados.

In [ ]:
# Importar as bibliotecas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler

Fazer a carga dos datasets a partir dos arquivos credtrain.txt e credtest.txt, passando o nome das colunas. Verificar se o dataset foi carregado corretamente e analisar o conteúdo do dataset

In [ ]:
# Define o nomes das colunas
nomes_colunas = ['ESCT', 'NDEP', 'RENDA', 'TIPOR', 'VBEM', 'NPARC', 'VPARC', 'TEL', 'IDADE', 'RESMS', 'ENTRADA', 'CLASSE']

# Carrega o dataset de treinamento
dsTrain = pd.read_csv('../data/credtrain.txt', names=nomes_colunas, sep='\t', header=0)

# Carrega o dataset de teste
dsTeste = pd.read_csv('../data/credtest.txt', names=nomes_colunas, sep='\t', header=0 )


In [ ]:
dsTrain.head()
dsTeste.head()
dsTrain.info()
dsTrain.describe()

No dataset apresentado é identificado alguns atributos categóricos, ainda que estejam representados numericamente, como: Estado Civil (ESCT), Tipo de residência (TIPOR) e se o cliente possui telefone (TEL). Nesses casos será aplicado a técnica de One-hot-encoding, que transpõe os domínios encontrados nestes atributos em campos nomeados destes valores. O campo de número de dependentes (NDEP), apesar de categório, apresenta-se de forma númerica e ordenada, característica coerente com a natureza deste atributo, sendo assim, não será necessário aplicar a técnica de One-hot-encoding em NDEP.

In [ ]:
dsTrain = pd.get_dummies(dsTrain, columns=['ESCT','TIPOR','TEL'])
dsTeste = pd.get_dummies(dsTeste, columns=['ESCT','TIPOR','TEL'])

dsTrain.head()
dsTeste.head()

O dataset é separado em features (X) e target (y) para o dataset de Treinamento (dsTrain) e de teste (dsTeste)

Para equalizar os dados de entrada (feature), foi aplicado a função StandardScaler.

In [ ]:
# Split the dataset into features (X) and the target variable (y)
X_train = dsTrain.drop('CLASSE', axis=1)
y_train = dsTrain['CLASSE']

X_test = dsTeste.drop('CLASSE', axis=1)
y_test = dsTeste['CLASSE']

# Scale the features using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


Uma vez realizado o pré-processamento dos dados, o modelo será treinado com os algoritmos solicitados. Verificando a acurácia de modelo.

Regressão Logística

In [ ]:
# Criar uma lista para os valores de acurácia e os modelos
listaModelos = ['Regressão Logística', 'Arvore de Decisão', 'Random Forest', 'K Neighbors', 'Gradient Boosting']
listaAcuracia =[]

# Aplicando a Regressão Logística
modelLR = LogisticRegression(solver='lbfgs', random_state=0)
modelLR.fit(X_train, y_train)

# Fazer a predição da Regressão Logística
y_predict_LR = modelLR.predict(X_test)


# Gerar a matriz de confusão
conf_matrix = confusion_matrix(y_test, y_predict_LR)

# Calcular a acurácia
accuracy = accuracy_score(y_test, y_predict_LR)
listaAcuracia.append(accuracy)

# Visualizar a matriz de confusão
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="BuGn", cbar=False)
plt.xlabel('Predito')
plt.ylabel('Real')
plt.title('Matriz de Confusão (Regressão Logística)')
plt.show()

print(f'Acurácia do modelo: {accuracy}')

Decision Tree Classifier

In [ ]:
# Aplicando a Decision Tree Classifier
modelTree = DecisionTreeClassifier(random_state=13)
modelTree.fit(X_train, y_train)

# Fazer a predição da Decision Tree Classifier
y_predict_Tree = modelTree.predict(X_test)

# Gerar a matriz de confusão
conf_matrix = confusion_matrix(y_test, y_predict_Tree)

# Calcular a acurácia
accuracy = accuracy_score(y_test, y_predict_Tree)
listaAcuracia.append(accuracy)

# Visualizar a Decision Tree Classifier
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", cbar=False)
plt.xlabel('Predito')
plt.ylabel('Real')
plt.title('Matriz de Confusão (Decision Tree Classifier)')
plt.show()

print(f'Acurácia do modelo: {accuracy}')



Random Forest Classifier

In [ ]:
# Aplicando a Random Forest Classifier
modelRF = RandomForestClassifier(n_estimators=2, random_state=0)
modelRF.fit(X_train, y_train)

# Fazer a predição da Random Forest Classifier
y_predict_RF = modelRF.predict(X_test)

# Gerar a matriz de confusão
conf_matrix = confusion_matrix(y_test, y_predict_RF)

# Calcular a acurácia
accuracy = accuracy_score(y_test, y_predict_RF)
listaAcuracia.append(accuracy)

# Visualizar a Random Forest Classifier
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", cbar=False)
plt.xlabel('Predito')
plt.ylabel('Real')
plt.title('Matriz de Confusão (Random Forest Classifier)')
plt.show()

print(f'Acurácia do modelo: {accuracy}')


K Neighbors Classifier

In [ ]:
# Aplicando a K Neighbors Classifier
modelKN = KNeighborsClassifier(n_neighbors=5)
modelKN.fit(X_train, y_train)

# Fazer a predição de K Neighbors Classifier
y_predict_KN = modelKN.predict(X_test)

# Gerar a matriz de confusão
conf_matrix = confusion_matrix(y_test, y_predict_KN)

# Calcular a acurácia
accuracy = accuracy_score(y_test, y_predict_KN)
listaAcuracia.append(accuracy)

# Visualizar a K Neighbors Classifier
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", cbar=False)
plt.xlabel('Predito')
plt.ylabel('Real')
plt.title('Matriz de Confusão (K Neighbor Classifier)')
plt.show()

print(f'Acurácia do modelo: {accuracy}')



Gradient Boosting Classifier

In [ ]:
# Aplicando a Gradient Boosting Classifier
modelGB = GradientBoostingClassifier()
modelGB.fit(X_train, y_train)

# Fazer a predição de Gradient Boosting Classifier
y_predict_GB = modelGB.predict(X_test)

# Gerar a matriz de confusão
conf_matrix = confusion_matrix(y_test, y_predict_GB)

# Calcular a acurácia
accuracy = accuracy_score(y_test, y_predict_GB)
listaAcuracia.append(accuracy)

# Visualizar a Gradiente Boosting Classifier
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", cbar=False)
plt.xlabel('Predito')
plt.ylabel('Real')
plt.title('Matriz de Confusão (Gradient Boosting Classifier)')
plt.show()

print(f'Acurácia do modelo: {accuracy}')



In [ ]:
# Modelo com melhor precisão
iMelhor=(listaAcuracia.index(max(listaAcuracia)))


print("O melhor modelo é", listaModelos[iMelhor], "com acurácia de", listaAcuracia[iMelhor])

# 2

Importar as bibliotecas necessárias (para efeito didático algumas bibliotecas serão carregadas novamente, as mesmas já carregadas no inicio do notebook) e carregar os dados em arquivo texto para o dataset

In [ ]:
# Importar as bibliotecas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder

Carregar os dados dos diamantes a partir do arquivo de texto. 

In [ ]:
# Carregar o dataset a partir do arquivo texto
dataset = pd.read_csv('../data/diamonds.csv', header=0)

dataset.info()

Os campos cut, color e clarity são objetos e devem ser convertidos em valores numéricos para processamento dos algoritmos. Estes campos obedecem uma ordem qualitativa, então foi utilizado o recurso Ordinal Encoder para fazer esta transformação. 

In [ ]:
#dataset.cut.unique()
dataset.clarity.unique()

# Determina um vetor com valores crescentes dos atributos cut, color e clarity
order_cut = ['Fair','Good', 'Very Good', 'Premium', 'Ideal']
order_color = ['J','I','H','G','F','E','D']
order_clarity = ['I3','I2','I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF','FL']

# Adiciona campo cut com numeraçao crescente
order_encoder_cut=OrdinalEncoder(categories=[order_cut])
cut_encoded = order_encoder_cut.fit_transform(dataset[['cut']])
dataset['cut_encoded'] = cut_encoded

# Adiciona campo color com numeraçao crescente
order_encoder_color=OrdinalEncoder(categories=[order_color])
color_encoded = order_encoder_color.fit_transform(dataset[['color']])
dataset['color_encoded'] = color_encoded

# Adiciona campo clarity com numeraçao crescente
order_encoder_clarity=OrdinalEncoder(categories=[order_clarity])
clarity_encoded = order_encoder_clarity.fit_transform(dataset[['clarity']])
dataset['clarity_encoded'] = clarity_encoded

# Remover as colunas de objetos que foram transformados
dataset = dataset.drop('cut', axis=1)
dataset = dataset.drop('color', axis=1)
dataset = dataset.drop('clarity', axis=1)

# Remove o campo identidade do ínico
dataset = dataset.drop(dataset.columns[0], axis=1) 
dataset.head()
dataset.drop(dataset.columns[0], axis=1)

# Remover os registos onde a dimensão do diamante é zero
dataset = dataset.drop(dataset[dataset["x"]==0].index)
dataset = dataset.drop(dataset[dataset["y"]==0].index)
dataset = dataset.drop(dataset[dataset["z"]==0].index)


dataset.describe()

E fazer a separação em dois datasets: treinamento e teste obedecendo a relação de 20% em relação ao total de registros. Para isto, será usado a biblioteca train_test_split. 

In [ ]:
# Define as features e o target do modelo (campo price) para predizer o valor
X= dataset.drop(["price"],axis =1)
y= dataset["price"]

# Separar em teste e treinamento na razão de 20%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

# Scale the features using StandardScaler
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Realizar o treinamento com cada um dos algoritmos de regressão listados: Regressão Linear, Lasso, Decison Tree Regressor, K Neighbors Regressor e Gradient Boosting Regressor. Como esses algoritmos de regressão compartilham as mesmas chamadas de fit, predict, r2_score e mean_squared_error, utilizando os hiperparâmetros padrões, será definida uma lista com as funções a serem executadas em bloco.

In [ ]:
# Definir a lista dos algoritmos a serem utilizados
listaAlgoritmosReg = [LinearRegression, Lasso, DecisionTreeRegressor, RandomForestRegressor, KNeighborsRegressor, GradientBoostingRegressor]

# Armazena o melhor algoritmo com base no valor do r2 (maior = melhor)
melhorR2 = 0
melhorMse = 0
melhorAlgoritmo = ""

for modeloAlgo in listaAlgoritmosReg:
    modeloSelecionado = modeloAlgo()
    modeloSelecionado.fit(X_train, y_train)
    y_pred_sel = modeloSelecionado.predict(X_test)

    mse = mean_squared_error(y_test, y_pred_sel)
    r2 = r2_score(y_test, y_pred_sel)

    if (r2 > melhorR2):
        melhorR2 = r2
        melhorMse= mse
        melhorAlgoritmo = modeloAlgo.__name__


    print(modeloAlgo.__name__)
    print("MSE = " ,mse)
    print('R2 = %.2f' % r2)
    print("--------------------------------------------------")

print("Melhor algoritmo = ", melhorAlgoritmo)
print("MSE = ", melhorMse)
print("R2 = ", melhorR2)

# 3

Como primeiro passo, vamos para a importação das bibliotecas necessárias.

In [62]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder

Fazer a importação dos dados a partir dos arquivos no formato pickle: A602.pickle, A621.pickle, A627.pickle, A636.pickle, A652.pickle. Depois unificar em um único dataset para análise

In [97]:
arquivos_pickle = ['A602','A621','A627','A636','A652']

flag = True
for arquivo in arquivos_pickle:
    caminho = '../data/atmoseer/'
    extensao = '.pickle'

    outfilename = caminho + arquivo + extensao
    print(outfilename)

    f = open(outfilename, 'rb')

    #Primeira ocorrencia do arquivo
    if (flag):
        (X_train , y_train , X_val , y_val , X_test , y_test ) = pickle.load(f) 
        flag = False
    else:
        (X_train_i , y_train_i , X_val_i , y_val_i , X_test_i , y_test_i ) = pickle.load(f) 

        # Adicionar conteúdo do arquivo pickle acumulando valores
        X_train = np.vstack((X_train, X_train_i))
        y_train = np.vstack((y_train, y_train_i))

        X_val = np.vstack((X_val, X_val_i))
        y_val = np.vstack((y_val, y_val_i))

        X_test = np.vstack((X_test, X_test_i))
        y_test = np.vstack((y_test, y_test_i))


# Converter as matrizes para dataframes
df_train = pd.DataFrame(X_train)
df_validation = pd.DataFrame(X_val)
df_test = pd.DataFrame(X_test)
df_train.describe()


../data/atmoseer/A602.pickle
../data/atmoseer/A621.pickle
../data/atmoseer/A627.pickle
../data/atmoseer/A636.pickle
../data/atmoseer/A652.pickle


,0,1,2,3,4,5,6,7,8,9,...,14,15,16,17,18,19,20,21,22,23
count,49640.000000,49640.000000,49640.000000,49640.000000,49640.000000,49640.000000,49640.000000,49640.000000,49640.000000,49640.000000,...,49640.000000,49640.000000,49640.000000,49640.000000,49640.000000,49640.000000,49640.000000,49640.000000,49640.000000,49640.000000
mean,0.421857,0.505501,0.747884,0.505829,0.471072,0.499833,0.495685,0.003897,0.422227,0.505237,...,0.495875,0.003891,0.422468,0.505024,0.746911,0.505727,0.471356,0.497698,0.496346,0.003880
std,0.154925,0.160470,0.181923,0.131709,0.149439,0.353127,0.353960,0.029061,0.155056,0.160375,...,0.353346,0.029044,0.155201,0.160298,0.182331,0.132072,0.149600,0.354302,0.352784,0.029017
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.313433,0.395918,0.638889,0.430825,0.353270,0.146447,0.146447,0.000000,0.313433,0.395918,...,0.146447,0.000000,0.313433,0.395918,0.638889,0.430387,0.353372,0.146447,0.146447,0.000000
50%,0.403042,0.500000,0.782051,0.495847,0.459387,0.500000,0.500000,0.000000,0.403448,0.500000,...,0.500000,0.000000,0.403448,0.497976,0.782051,0.495806,0.459387,0.500000,0.500000,0.000000
75%,0.513793,0.607287,0.888889,0.571563,0.572553,0.853553,0.853553,0.000000,0.516729,0.607287,...,0.853553,0.000000,0.516854,0.605691,0.888889,0.571992,0.572898,0.853553,0.853553,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
